In [175]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import requests
from tqdm import tqdm
import numpy as np
import time

In [167]:
BASE_URL = "https://openpostcode.nl/api/address"

params = {
    "postcode": "5056TG",
    "huisnummer": '3'
}

response = requests.get(BASE_URL, params=params)

In [168]:
response.json()

{'postcode': '5056TG',
 'huisnummer': '3',
 'straat': 'Erasmuslaan',
 'buurt': 'Eikenbosch',
 'wijk': 'Berkel-Enschot',
 'woonplaats': 'Berkel-Enschot',
 'gemeente': 'Tilburg',
 'provincie': 'Noord-Brabant',
 'latitude': 51.585545,
 'longitude': 5.147606}

In [169]:
table_name = 'postcode_info_NL'

In [179]:
response = supabase.table(table_name).select("*").execute()
postcode_database = response.data

In [180]:
postcode_database

[{'id': 1,
  'created_at': '2025-08-18T09:52:14.325927+00:00',
  'post_code': '0000AA',
  'latitude': None,
  'longitude': None}]

In [183]:
import string
import itertools

# Generate digits (4 digits, zero-padded)
digit_combos = [f"{i:04d}" for i in range(10000)]

# Generate letter combos (AA to ZZ)
letter_combos = [''.join(pair) for pair in itertools.product(string.ascii_uppercase, repeat=2)]

# Loop through all combinations (generator approach)
for digits in tqdm(digit_combos):
    for letters in letter_combos:
        code = digits + letters
        # print(code)  # or process it
        params = {
            "postcode": code,
            "huisnummer": 1
        }
        response = requests.get(BASE_URL, params=params)
        if 'latitude' and 'longitude' in response.json().keys():
            lat = response.json()['latitude']
            lon = response.json()['longitude']
        elif response.json()['error'] == 'Huisnummer not found':
            params = {
                "postcode": code,
                "huisnummer": response.json()['suggestions'][0]
            }
            response = requests.get(BASE_URL, params=params)
            lat = response.json()['latitude']
            lon = response.json()['longitude']
        else:
            lat = None
            lon = None
        supabase_response = (
            supabase.table(table_name)
            .insert({"post_code": code, "longitude": lon, "latitude": lat})
            .execute()
        )
        time.sleep(0.01)
            

  0%|▏                                                                                                                                         | 13/10000 [22:48<292:01:10, 105.26s/it]


ConnectionError: ('Connection aborted.', ConnectionAbortedError(10053, 'An established connection was aborted by the software in your host machine', None, 10053, None))

In [173]:
code = '0000AA'

In [181]:
# response = (
#     supabase.table("scraped_data")
#     .update({"lat_postcode": 0})
#     .eq("car_id", 'a835a871-370a-4ce2-9250-26b64bb928ea	')
#     .execute()
# )

# response = (
#     supabase.table("scraped_data")
#     .update({"lon_postcode": 0})
#     .eq("car_id", 'a835a871-370a-4ce2-9250-26b64bb928ea	')
#     .execute()
# )

response = (
    supabase.table(table_name)
    .insert({"post_code": code, "longitude": None, "latitude": None})
    .execute()
)